In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras import backend as K

import matplotlib.pyplot as plt
import scipy.misc

In [ ]:
img_width = 160
img_height = 120
image_size=(img_height, img_width)
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_height, img_width)
else:
    input_shape = (img_height, img_width, 3)
    
print(input_shape)
nb_train_samples = 10000
nb_validation_samples = 4000
# Path to datadir where train, val and test directories reside
datadir = 'Jun21'

batch_size = 20
nb_angles = 15

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        datadir + '/train',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')

#Validation Generator
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
        datadir + '/val',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')

print(train_generator.class_indices)
print(val_generator.class_indices)

In [ ]:
# Python2 uses iteritems() while Python3 uses items()
inv_map = {v: k for k, v in train_generator.class_indices.items()}

In [ ]:
nb_epoch = 50
nb_filters=16
kernel_size=(3,3)
pool_size=(2,2)
#img = Input(shape=input_shape)

In [ ]:
idx = '1'
model_file = 'models/out15_' + datadir + '_' + idx + '.h5'
weights_file = 'weights/out15_' + datadir + '_' + idx + '.h5'
got_weights = False
save_model = True
try:
    model = load_model(model_file)
    print('Model loaded')
    got_weights = True
    save_model = False
except:
    model = Sequential()
    model.add(Conv2D(nb_filters, kernel_size, input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Conv2D(2*nb_filters, kernel_size))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Conv2D(2*nb_filters, kernel_size))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Conv2D(4*nb_filters, kernel_size))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(nb_angles, activation = 'softmax', name = 'angle_out'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

    print('Model compiled')
    try:
        model.load_weights(weights_file)
        got_weights = True
        print('Weights loaded')
    except:
        got_weights = False

print(got_weights)
model.summary()

In [ ]:
hist = None
if not got_weights:
    steps_pe = 200
    valSteps = 50
    hist = model.fit_generator(train_generator, steps_per_epoch=steps_pe,
              epochs=nb_epoch, validation_data=val_generator,
              validation_steps=valSteps)

    model.save_weights(weights_file)
    save_model = True
    print('Model trained and weights saved')

if save_model:
    model.save(model_file)
    print('Model saved')

In [ ]:
score = model.evaluate_generator(val_generator, steps=100)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
if hist:
    plt.close()
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.show()
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.show()

In [ ]:
#Test generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        datadir + '/test',
        target_size=image_size,
        batch_size=20,
        class_mode=None)
print(test_generator.class_indices)

In [ ]:
notDone = True
i = 0
axis_font = {'size':'48'}
while(notDone):
  batch_X = test_generator.next()
  #print(batch_X)
  batch_y = model.predict_on_batch(batch_X)
  #print(batch_y)
  j = 0
  f, axarr = plt.subplots(5, 4, figsize=(80, 60))
  for (X, y) in zip(batch_X, batch_y):
    #ax = plt.subplot(gl[j, i])
    k = j % 5
    l = j // 5
    #print(k, l)
    axarr[k, l].imshow(X)
    idx = np.argmax(y)
    txt = inv_map[idx] + ', ' + str(round(y[idx], 2))
    axarr[k, l].text(0, 0, txt, **axis_font)
    j += 1
    if j >= 20:
      break
    #print(y)
    #pause
  i += 1
  notDone = i < 5
  plt.show()


In [ ]:
print(inv_map)